In [143]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

In [144]:
filepath = 'C:/Users/phill/INFO698-Capstone/Anonymized Dataset for Info Science MS.csv'
data = pd.read_csv(filepath)

In [145]:
# Drop all records that do not include a Term GPA, as this will be our response variable for all models
data = data.dropna(subset = ['TermGPA'])

In [98]:
# Saving code that found outliers in the data set, but ones that we ultimately decided to keep in the data
# data.loc[(data['UnitsPassedincludedinGPA'] == 0) & (data['UnitsPassednotincludedinGPA'] == 0) & (data['TermGPA'] > 0)]
# data.loc[(data['UnitsPassedincludedinGPA'] == 0) & (data['UnitsPassednotincludedinGPA'] > 0)]
# data.loc[(data['UnitsPassedincludedinGPA'] == 0) & (data['UnitsPassednotincludedinGPA'] == 0) & (data['TermGPA'] == 0)]

In [146]:
# Group ages into ranges for categorical scoring
data['AgeGroups'] = pd.cut(data['Age'], bins=[0,24,39,59,100], labels=['<25','25-39','40-59','60+'])
# data['AgeGroups'].value_counts()

In [147]:
# Create a weighted GPA value to give more weight to those taking more classes (may not use this)
data['WeightedGPA'] = data['TermGPA'] * data['UnitsPassedincludedinGPA']
# data[['TermGPA', 'UnitsPassedincludedinGPA', 'WeightedGPA']]

In [148]:
# Create zscore values for each categorical field to feed into the model in addition to one-hot encoding of membership in each group
data['zFirstGen'] = data.groupby('FirstGenerationFlag', observed=True)['TermGPA'].transform(zscore)
data['zAge'] = data.groupby('AgeGroups', observed=True)['TermGPA'].transform(zscore)
data['zGender'] = data.groupby('Gender', observed=True)['TermGPA'].transform(zscore)
data['zMilitary'] = data.groupby('PrimaryMilitaryAffiliation', observed=True)['TermGPA'].transform(zscore)
data['zCollege'] = data.groupby('College', observed=True)['TermGPA'].transform(zscore)
data['zAcademicCareer'] = data.groupby('AcademicCareer', observed=True)['TermGPA'].transform(zscore)
data['zAcademicLevel'] = data.groupby('AcademicLevelEndofTerm', observed=True)['TermGPA'].transform(zscore)
data['zFullTimePartTime'] = data.groupby('UAFullTimePartTime', observed=True)['TermGPA'].transform(zscore)
data['zClassesEnrolled'] = data.groupby('NumberofClassesEnrolled', observed=True)['TermGPA'].transform(zscore)

In [150]:
data

,FakeIdentifier,Age,Gender,PrimaryMilitaryAffiliation,AcademicCareer,College,NumberofClassesEnrolled,TermGPA,CumulativeGPA,UnitsPassedincludedinGPA,...,WeightedGPA,zFirstGen,zAge,zGender,zMilitary,zCollege,zAcademicCareer,zAcademicLevel,zFullTimePartTime,zClassesEnrolled
0,1,18,Female,No Military Affiliation,Undergraduate,College of Science,4,3.75,3.64,4,...,15.00,0.386971,0.652534,0.486285,0.483992,0.675598,0.612251,0.813409,0.484412,0.528375
1,2,72,Male,No Military Affiliation,Law,James E Rogers College of Law,3,4.00,3.83,3,...,12.00,0.622786,0.577925,0.685833,0.696447,0.618656,0.619093,0.619093,0.726870,0.692689
3,4,47,Female,Veteran,Graduate,Coll of Ag Life & Env Sci,2,4.00,4.00,3,...,12.00,0.622786,0.570341,0.697668,0.671944,0.776177,0.444102,0.444102,0.687788,0.642589
4,5,45,Male,No Military Affiliation,Undergraduate,College of Information Science,2,2.00,3.25,3,...,6.00,-0.746523,-1.312691,-1.046009,-1.003195,-1.106019,-0.764314,-0.523431,-0.939218,-1.058836
5,6,22,Female,No Military Affiliation,Graduate,College of Social & Behav Sci,3,4.00,4.00,9,...,36.00,0.622786,0.852595,0.697668,0.696447,0.723399,0.444102,0.444102,0.726870,0.692689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66423,66424,28,Female,Guard Reserve,Undergraduate,Coll of Ag Life & Env Sci,3,4.00,4.00,3,...,12.00,0.806605,0.652378,0.697668,0.757997,0.776177,0.808903,0.830265,0.687788,0.692689
66424,66425,47,Female,No Military Affiliation,Graduate,Zuckerman Coll Public Health,3,4.00,3.88,3,...,12.00,0.622786,0.570341,0.697668,0.696447,0.572340,0.444102,0.444102,0.687788,0.692689
66426,66427,43,Male,No Military Affiliation,Graduate,Eller College of Management,2,4.00,3.80,6,...,24.00,0.622786,0.570341,0.685833,0.696447,0.773944,0.444102,0.444102,0.687788,0.642589
66427,66428,46,Female,No Military Affiliation,Law,James E Rogers College of Law,3,3.67,3.57,9,...,33.03,0.311511,0.259641,0.418642,0.416006,0.232805,0.233363,0.233363,0.388198,0.407529
